### **Libraries**

Some of the needed libraries for this project.

In [19]:
import os
import openai
from openai import OpenAI

In [20]:
import pathlib
import textwrap

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [21]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") # OpenAI API key

#### **Define a client**

Created an instance of the OpenAI model

In [22]:
# define an instance of OpenAI model
client = OpenAI(api_key=OPENAI_API_KEY)

#### **Audio transcription**

Transcribed the audio file.

In [23]:
audio_file= open("multiple.mp3", "rb")

In [24]:
speech_to_text = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file
)

In [25]:
to_markdown(speech_to_text.text)

> On the 21st of July 2024, I'll be having a meeting with Alex, which will start by 7 p.m. and end by 8 p.m. The event will take place on Google Meet. Set the reminder for 15 minutes. And on the 28th of July 2024, I'll have another meeting with Adam by 3 p.m. and it will end by 4 p.m. The event is going to take place on Google Meet. Set the reminder for 10 minutes.

### **Conversational chat**

Using conversational chat to extract event related data.

In [35]:
extract_prompt = f"""
You are an assistant that creates a well crafted schedule from text. 
Extract all events related data from the provided text and put it in JSON format.


{speech_to_text.text}
"""

In [27]:
from langchain_openai import ChatOpenAI

In [28]:
chatmodel = ChatOpenAI(
    model='gpt-3.5-turbo',
    api_key=OPENAI_API_KEY,
    temperature=0.3,
)

In [36]:
format_chat  = chatmodel.invoke(extract_prompt)


In [30]:
to_markdown(format_chat.content)

> {
>   "events": [
>     {
>       "date": "21st of July 2024",
>       "event": "Meeting with Alex",
>       "start_time": "7:00 p.m.",
>       "end_time": "8:00 p.m.",
>       "location": "Google Meet",
>       "reminder": "15 minutes"
>     },
>     {
>       "date": "28th of July 2024",
>       "event": "Meeting with Adam",
>       "start_time": "3:00 p.m.",
>       "end_time": "4:00 p.m.",
>       "location": "Google Meet",
>       "reminder": "10 minutes"
>     }
>   ]
> }

#### **Convert the datatype**

In [70]:
data_type = f"""
I want you to convert the date and time from the text to a format that can be easily converted
Put all time in 24 hours format.


Example:
21st of July 2024 >> 2024-07-21
10 minutes >> 00:10

and parse the output as a JSON object.


{format_chat.content}
"""

In [71]:
data_chat = chatmodel.invoke(data_type)
to_markdown(data_chat.content)

> {
>   "events": [
>     {
>       "date": "2024-07-21",
>       "event": "Meeting with Alex",
>       "start_time": "19:00",
>       "end_time": "20:00",
>       "location": "Google Meet",
>       "reminder": "00:15"
>     },
>     {
>       "date": "2024-07-28",
>       "event": "Meeting with Adam",
>       "start_time": "15:00",
>       "end_time": "16:00",
>       "location": "Google Meet",
>       "reminder": "00:10"
>     }
>   ]
> }

#### **String output to JSON file**

In [72]:
# Used to create a JSON file
import json

In [73]:
# converts string to dictionary
result_dict = json.loads(data_chat.content)

In [74]:

# Define the file path
json_file_path = 'events.json'

# Write the result to the JSON file
with open(json_file_path, 'w') as file:
    json.dump(result_dict, file, indent=4)

#### **JSON file to .ics file**

In [75]:
import json  # Import json module to read JSON file
from icalendar import Calendar, Event, Alarm
from datetime import datetime, timedelta
from pytz import UTC  # This is required for the 'dt' fields in icalendar

# Path to your JSON file
json_file_path = 'events.json'

# Read and load the JSON data from file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Create an iCalendar file
cal = Calendar()

for event in data['events']:
    # Create an event
    ical_event = Event()
    
    # Correctly access the properties of each event
    start_date_str = event['date'] + ' ' + event['start_time']
    end_date_str = event['date'] + ' ' + event['end_time']
    reminder_str = event['date'] + ' ' + event['reminder']
    
    # Convert to datetime objects
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M').replace(tzinfo=UTC)
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M').replace(tzinfo=UTC)
    reminder_time = datetime.strptime(reminder_str, '%Y-%m-%d %H:%M').replace(tzinfo=UTC)
    
    # Calculate reminder trigger time as a timedelta before the event start
    reminder_delta = start_date - reminder_time
    
    # Add details to the event
    ical_event.add('summary', event['event'])
    ical_event.add('dtstart', start_date)
    ical_event.add('dtend', end_date)
    
    # Optional: Add location and description if available
    if 'location' in event:
        ical_event.add('location', event['location'])
    if 'event_details' in event:
        ical_event.add('description', event.get('event_details', ''))

    # Create a reminder (alarm)
    alarm = Alarm()
    alarm.add('action', 'DISPLAY')
    alarm.add('description', 'Reminder')
    alarm.add('trigger', -reminder_delta)  
    
    # Add the alarm to the event
    ical_event.add_component(alarm)
    
    # Add the event to the calendar
    cal.add_component(ical_event)

# Save the iCalendar file
file_name = 'events.ics'
with open(file_name, 'wb') as f:
    f.write(cal.to_ical())

print(f"iCalendar file '{file_name}' created successfully.")

iCalendar file 'events.ics' created successfully.


#### **Writing the .ics file to calendar**

In [76]:
import os
from abc import ABC, abstractmethod
from icalendar import Calendar

# Step 1: Read and parse the ICS file
def read_ics(file_name):
    with open(file_name, 'r') as ics_file:
        return Calendar.from_ical(ics_file.read())

# Step 2: Define an abstract base class for calendar services
class CalendarService(ABC):
    def __init__(self, credentials):
        self.credentials = credentials

    @abstractmethod
    def authenticate(self):
        pass

    @abstractmethod
    def create_event(self, event):
        pass

# Step 3: Implement Google Calendar handler
class GoogleCalendarService(CalendarService):
    def authenticate(self):
        from google.oauth2.credentials import Credentials
        from google_auth_oauthlib.flow import InstalledAppFlow
        from google.auth.transport.requests import Request
        from googleapiclient.discovery import build

        SCOPES = ['https://www.googleapis.com/auth/calendar']
        
        creds = None
        if os.path.exists('token.json'):
            creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials, SCOPES)
                creds = flow.run_local_server(port=0)
            with open('token.json', 'w') as token:
                token.write(creds.to_json())
        
        self.service = build('calendar', 'v3', credentials=creds)

    def create_event(self, event):
        google_event = {
            'summary': str(event.get('summary')),
            'start': {'dateTime': event.get('dtstart').dt.isoformat(), 'timeZone': 'UTC'},
            'end': {'dateTime': event.get('dtend').dt.isoformat(), 'timeZone': 'UTC'}
        }
        self.service.events().insert(calendarId='primary', body=google_event).execute()

# Step 5: General function to add events to calendar
def add_events_to_calendar(service, ics_file_path):
    calendar = read_ics(ics_file_path)
    for component in calendar.walk():
        if component.name == "VEVENT":
            service.create_event(component)

# Example Usage:
# Initialize the Google Calendar service
google_service = GoogleCalendarService('credential.json')
google_service.authenticate()
add_events_to_calendar(google_service, 'events.ics')